 # JPDB Project: Database Connection, Schema, and Data Ingestion (Commented)

In [10]:
import pandas as pd
import json
import os
import time, random
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Boolean, DateTime, Text, ForeignKey, insert, select, update
from jpdb_functions import create_prebuilt, get_decks, get_vocab, lookup_vocab, replace_empty_values, delete_deck


 ## Connect to Database and Reflect Tables

In [11]:
engine = create_engine('sqlite:///jpdb_project.db')
metadata = MetaData()

novels_table = Table('novels', metadata, autoload_with=engine)
vocab_table = Table('vocab', metadata, autoload_with=engine)
novel_vocab_table = Table('novel_vocab', metadata, autoload_with=engine)
anki_vocab_table = Table('anki_vocab', metadata, autoload_with=engine)


 ## Confirm Table Columns

In [12]:
print(novels_table.columns.keys())
print(vocab_table.columns.keys())
print(novel_vocab_table.columns.keys())
print(anki_vocab_table.columns.keys())


['novel_id', 'Title', 'Length (in words)', 'Unique words', 'Unique words (used once)', 'Unique kanji', 'Unique kanji (used once)', 'Unique kanji readings', 'Average difficulty', 'Peak difficulty (90th percentile)', 'Average sentence length', 'Characters', 'Volumes', 'Blacklisted unique words']
['vid', 'sid', 'spelling', 'reading', 'frequency_rank', 'meanings', 'blacklisted']
['novel_id', 'vid']
['id', 'vocab', 'reading', 'vid', 'sid', 'frequency_rank', 'percent_correct', 'due', 'again_count', 'reviews']


 ## Preview Data

In [13]:
with engine.connect() as conn:
    result = conn.execute(select(novels_table).limit(5))
    display(pd.DataFrame(result.fetchall(), columns=result.keys()))

with engine.connect() as conn:
    result = conn.execute(select(vocab_table).limit(20))
    display(pd.DataFrame(result.fetchall(), columns=result.keys()))

with engine.connect() as conn:
    result = conn.execute(select(anki_vocab_table).limit(5))
    display(pd.DataFrame(result.fetchall(), columns=result.keys()))

with engine.connect() as conn:
    result = conn.execute(select(novel_vocab_table).limit(5))
    display(pd.DataFrame(result.fetchall(), columns=result.keys()))


,novel_id,Title,Length (in words),Unique words,Unique words (used once),Unique kanji,Unique kanji (used once),Unique kanji readings,Average difficulty,Peak difficulty (90th percentile),Average sentence length,Characters,Volumes,Blacklisted unique words
0,1587,Classroom of the Elite,749776,19931,6179,2282,216,4527,39,49,20.4,1757593,14,658
1,1588,Fate/Zero (TYPE-MOON BOOKS ver.),272859,19312,7084,2780,296,5582,89,96,31.1,620439,4,490
2,1589,Baka to Tesuto to Shoukanjuu,397303,17018,6329,2305,273,4487,22,38,19.1,936627,11,614
3,1590,Golden Time,211425,14969,6335,2172,281,4017,62,72,21.2,498836,5,649
4,1591,Fragments of the Stars,103075,10208,4553,2106,327,3654,75,90,23.9,246367,2,161


,vid,sid,spelling,reading,frequency_rank,meanings,blacklisted
0,1000000,2204407130,ヽ,ヽ,79400,"[""repetition mark in katakana""]",False
1,1000010,2204406695,ヾ,ヾ,103500,"[""voiced repetition mark in katakana""]",False
2,1000020,2204190935,ゝ,ゝ,127200,"[""repetition mark in hiragana""]",False
3,1000030,2204191370,ゞ,ゞ,193200,"[""voiced repetition mark in hiragana""]",False
4,1000040,2204681615,〃,おなじ,81300,"[""ditto mark""]",False
5,1000060,1999181481,ノマ,ノマ,158000,"[""kanji repetition mark""]",False
6,1000090,2289457895,○,まる,8300,"[""circle"", ""\""correct\""; \""good\"""", ""*; _"", ""p...",False
7,1000100,981258553,ＡＢＣ順,エービーシーじゅん,198100,"[""alphabetical order""]",False
8,1000110,3138583194,ＣＤプレイヤー,シーディープレイヤー,106800,"[""CD player""]",False
9,1000140,3107122560,Ｏバック,オーバック,201400,"[""O-back; skirt with peek-a-boo hole in rump""]",False


,id,vocab,reading,vid,sid,frequency_rank,percent_correct,due,again_count,reviews
0,0,起こる,おこる,1223680,2146305150,800,100.0,2025-06-18,1,13
1,1,重要,じゅうよう,1336820,3870032381,1200,94.0,2025-06-18,2,16
2,2,至る,いたる,1311870,1536129181,1300,100.0,2025-06-18,1,13
3,3,対応,たいおう,1409840,3337990295,1300,88.0,2025-06-18,5,33
4,4,何より,なにより,1188530,3589014081,1100,100.0,2025-06-18,0,10


,novel_id,vid
0,1587,1543220
1,1587,1162940
2,1587,1310530
3,1587,1169090
4,1587,1902070


 ## (Commented) Insert Cleaned Novels Data

In [14]:
# df = pd.read_csv('jpdb_novel_data_cleaned.csv')
# with engine.begin() as conn:
#     values_list = []
#     for index, row in df.iterrows():
#         novel_dict = {
#             "novel_id": row['novel_id'],
#             "Title": row['Title'],
#             "Length (in words)": row['Length (in words)'],
#             "Unique words": row['Unique words'],
#             "Unique words (used once)": row['Unique words (used once)'],
#             'Unique kanji': row['Unique kanji'],
#             'Unique kanji (used once)': row['Unique kanji (used once)'],
#             'Unique kanji readings': row['Unique kanji readings'],
#             "Average difficulty": int(row['Average difficulty'].split('/')[0]),
#             "Peak difficulty (90th percentile)": int(row['Peak difficulty (90th percentile)'].split('/')[0]),
#             "Average sentence length": row['Average sentence length'],
#             "Characters": row['Characters'],
#             "Volumes": row['Volumes'],
#             "Blacklisted unique words": row['Blacklisted unique words']
#         }

#         values_list.append(novel_dict)
        
#     stmt = insert(novels_table)
#     result_proxy = conn.execute(stmt, values_list)
#     print(result_proxy.rowcount) #Inserted 1475 rows

 ## (Commented) Insert Anki Vocab Data

In [15]:
# anki_df = pd.read_csv('anki_df.csv')
# with engine.begin() as connection:
#     insert_stmt = insert(anki_vocab)
#     anki_vocab_value_list = []
#     for index, row in anki_df.iterrows():
#         anki_vocab_dict = {
#             "id": index,
#             "vocab": row['Vocab'],
#             "reading": row['Reading'],
#             "vid": row['vid'],
#             "sid": row['sid'],
#             "frequency_rank": row['frequency rank'],
#             "percent_correct": row['Percent Correct'],
#             "again_count": row['Again Count'],
#             "due": row['Due'],
#             "reviews": row['Reviews']
#         }
    
#         anki_vocab_value_list.append(anki_vocab_dict)
    
#     connection.execute(insert_stmt, anki_vocab_value_list)




## (Commented) Insert vocab data (from novels)

In [16]:
## Adding vocab data to db ##
# with engine.begin() as connection:
#     select_stmt = select(novels)
#     result_proxy = connection.execute(select_stmt).fetchall()
#     current_novel_index = 1299

#     #Create a set to track which vids (vocab id) have been seen
#     select_vid_stmt = select(vocab.c.vid)
#     vid_results = connection.execute(select_vid_stmt)
#     seen_vids = set(row[0] for row in vid_results)

#     for entry in result_proxy[0:]:
#         current_novel_index += 1
#         novel_id = entry[0]
#         novel_name = entry[1]
        
        
#         #Create the prebuilt deck with vocab in jpdb account
#         create_prebuilt(novel_id=novel_id, sid=SID)
#         deck_data = get_decks()['decks'][0]
#         user_deck_id = deck_data[0]

#         deck_vocab_list = get_vocab(user_deck_id=user_deck_id)
#         vocab_data = lookup_vocab(deck_vocab_list)

#         vocab_value_list = []
#         novel_vocab_value_list = []
#         for vocab_entry in vocab_data:
            
#             vid = vocab_entry[0]

#             novel_vocab_dict = {
#                 "novel_id": novel_id,
#                 "vid": vid
#             }

#             novel_vocab_value_list.append(novel_vocab_dict)
            
#             if vid in seen_vids:
#                 continue
            
#             seen_vids.add(vid)

#             replace_empty_values(vocab_entry)
#             vocab_dict = {
#                 "vid": vid,
#                 "sid": vocab_entry[1],
#                 "spelling": vocab_entry[2],
#                 "reading": vocab_entry[3],
#                 "frequency_rank": int(vocab_entry[4]),
#                 "meanings": json.dumps(vocab_entry[5])
#             }
            
            
#             vocab_value_list.append(vocab_dict)
        
#         #Checks if vocab_value list is empty to prevent inserting empty list into table
#         if len(vocab_value_list) != 0:
#             vocab_insert_stmt = insert(vocab)
#             vocab_result_proxy = connection.execute(vocab_insert_stmt, vocab_value_list)

#         novel_vocab_insert_stmt = insert(novel_vocab)
#         novel_vocab_result_proxy = connection.execute(novel_vocab_insert_stmt, novel_vocab_value_list)

        
#         time.sleep(random.uniform(1, 3))
#         delete_deck(user_deck_id)
#         print(f"Vocab for {novel_name} added to table. Novel index {current_novel_index} done.")

# print("Done with updates")

## (Commented) Updating new column blacklisted in vocab with False values for those not blacklisted.

In [17]:
# with engine.begin() as conn:
#     update_stmt = update(vocab).where(vocab.c.blacklisted == None).values(blacklisted = False)
#     conn.execute(update_stmt)

# with engine.begin() as conn:
#     anki_select_stmt = select(anki_vocab.c.vid)
#     anki_vids = conn.execute(anki_select_stmt).fetchall()


#     vocab_select_stmt = select(vocab.c.vid)
#     vocab_vids = conn.execute(vocab_select_stmt).fetchall()
   

#     i = 0
#     for vid in anki_vids:
#         if vid in vocab_vids:
#             i += 1
#     print(i) #3868 anki vocab inside vocab table out of 3876 anki vocab